In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
import pickle
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from keras import models
from keras.models import Sequential
from keras.optimizers import Adam
import tensorflow as tf
import os
import copy

In [2]:
# Run Noise_process before run this file
def get_dataset(obj_name):
    filename_labels = 'train_labels.npy'
    filename_origin = 'origin_cifar10.npy'
    filename_small_noise = 'small_noise_cifar10.npy'
    filename_medium_noise = 'medium_noise_cifar10.npy'
    filename_large_noise = 'large_noise_cifar10.npy'
    
    train_set = None
    match obj_name:
        case 'origin':
            train_set = np.load(filename_origin)
        case 'small_noise':
            train_set = np.load(filename_small_noise)
        case 'medium_noise':
            train_set = np.load(filename_medium_noise)
        case 'large_noise':
            train_set = np.load(filename_large_noise)
        case _:
            raise ValueError(f"Unknown obj_name: {obj_name}")
    label_set = np.load(filename_labels) if train_set.size > 0 else None

    if label_set is None or train_set is None:
        print("Run Noise_process.ipynb before run this file")
    return train_set, label_set

In [3]:
def build_simple_cnn(num_filter, learning_rate=0.0005, filter_size=(3,3), input_shape=(32, 32, 3), num_classes=10):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))  
    model.add(layers.Conv2D(num_filter, filter_size, activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [4]:
def get_processed_dataset(model_name):
    train_set, label_set = get_dataset(model_name)

    X_combined_reshaped = train_set.reshape(-1, 32, 32, 3)
    X_combined_reshaped = X_combined_reshaped.astype('float32') / 255.0
    y_combined_categorical = to_categorical(label_set, num_classes=10)

    return X_combined_reshaped, y_combined_categorical

In [5]:
ORIGIN='origin'
SMALL='small_noise'
MEDIUM='medium_noise'
LARGE='large_noise'
all_dataset = [ORIGIN, SMALL, MEDIUM, LARGE]

batch_size = 128
num_epochs = 5

In [6]:
def get_processed_dataset(model_name):
    train_set, label_set = get_dataset(model_name)

    X_combined_reshaped = train_set.reshape(-1, 32, 32, 3)
    X_combined_reshaped = X_combined_reshaped.astype('float32') / 255.0
    y_combined_categorical = to_categorical(label_set, num_classes=10)

    return X_combined_reshaped, y_combined_categorical

In [7]:
num_filter_configs = [8, 16, 32, 64, 128]

In [8]:
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)
best_params = {}
performance_results = []

x_combined, y_combined = get_processed_dataset(LARGE)

for outer_train_idx, outer_val_idx in outer_cv.split(x_combined):
    X_outer_train, X_outer_val = x_combined[outer_train_idx], x_combined[outer_val_idx]
    y_outer_train, y_outer_val = y_combined[outer_train_idx], y_combined[outer_val_idx]

    best_score = -np.inf
    best_num_filter = None

    for num_filter in num_filter_configs:
        inner_cv = KFold(n_splits=4, shuffle=True, random_state=42)
        scores = []
        
        for inner_train_idx, inner_val_idx in inner_cv.split(X_outer_train):
            X_inner_train, X_inner_val = X_outer_train[inner_train_idx], X_outer_train[inner_val_idx]
            y_inner_train, y_inner_val = y_outer_train[inner_train_idx], y_outer_train[inner_val_idx]
            
            model = build_simple_cnn(num_filter)
            history = model.fit(
                x=X_inner_train,
                y=y_inner_train,
                batch_size=batch_size,
                epochs=num_epochs,
                validation_data=(X_inner_val, y_inner_val)
            )
            
            score = model.evaluate(X_inner_val, y_inner_val, verbose=0)[0] 
            scores.append(score)

        mean_score = np.mean(scores)
        
        if mean_score > best_score:
            best_score = mean_score
            best_num_filter = num_filter

    best_params[outer_val_idx[0]] = best_num_filter
    final_model = build_simple_cnn(best_num_filter)
    final_model.fit(X_outer_train, y_outer_train, epochs=5, batch_size=128, verbose=0)
    final_performance = final_model.evaluate(X_outer_val, y_outer_val)[1]
    print("final_performance: ", final_performance)
    performance_results.append(final_performance)

average_performance = np.mean(performance_results)
print(f'Average Performance across all outer folds: {average_performance}')
print(f'Best Parameters for each fold: {best_params}')

Epoch 1/5
29/29 [==============================] - 15s 273ms/step - loss: 2.0720 - accuracy: 0.2503 - val_loss: 2.1970 - val_accuracy: 0.1742
Epoch 2/5
29/29 [==============================] - 3s 101ms/step - loss: 1.6842 - accuracy: 0.4050 - val_loss: 2.1595 - val_accuracy: 0.2100
Epoch 3/5
29/29 [==============================] - 3s 100ms/step - loss: 1.5312 - accuracy: 0.4567 - val_loss: 2.1077 - val_accuracy: 0.2992
Epoch 4/5
29/29 [==============================] - 2s 67ms/step - loss: 1.3987 - accuracy: 0.5147 - val_loss: 2.0588 - val_accuracy: 0.3133
Epoch 5/5
29/29 [==============================] - 1s 49ms/step - loss: 1.2699 - accuracy: 0.5644 - val_loss: 2.0197 - val_accuracy: 0.3467
Epoch 1/5
29/29 [==============================] - 7s 63ms/step - loss: 2.0882 - accuracy: 0.2719 - val_loss: 2.2030 - val_accuracy: 0.1892
Epoch 2/5
29/29 [==============================] - 2s 66ms/step - loss: 1.7140 - accuracy: 0.3989 - val_loss: 2.1403 - val_accuracy: 0.2583
Epoch 3/5
29/29 